# Final Project for INFO 6350
## Net ID: yz2685

In [1]:
# import our libraries
import requests
import urllib
from bs4 import BeautifulSoup
import simplejson as json
from urllib.request import Request, urlopen
from fake_useragent import UserAgent

In [2]:
# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
content = requests.get(documents_url, headers=header)
print(content)

<Response [200]>
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/FilingSummary.xml


# Grabbing the Financial Statements

In [6]:
# create the list to hold the statement urls
statements_url = []

for report_dict in master_reports:
    
    # define the statements we want to look for.
    item1 = r"Consolidated Balance Sheets"
    item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
    item3 = r"Consolidated Statements of Cash Flows"
    item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
    # store them in a list.
    report_list = [item1, item2, item3, item4]
    
    # if the short name can be found in the report list.
    if report_dict['name_short'] in report_list:
        
        # print some info and store it in the statements url.
        print('-'*100)
        print(report_dict['name_short'])
        print(report_dict['url'])
        
        statements_url.append(report_dict['url'])

----------------------------------------------------------------------------------------------------
Consolidated Balance Sheets
https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/R2.htm
----------------------------------------------------------------------------------------------------
Consolidated Statements of Operations and Comprehensive Income (Loss)
https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/R4.htm
----------------------------------------------------------------------------------------------------
Consolidated Statements of Cash Flows
https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/R5.htm
----------------------------------------------------------------------------------------------------
Consolidated Statements of Stockholder's (Deficit) Equity
https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/R6.htm


# Scraping the Financial Statements

# Scraping the SEC Query Page

In [7]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'1318605',
              'type':'10-k',
              'dateb':'20230101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers=header)
# response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1318605&type=10-k&dateb=20230101&owner=exclude&start=&output=&count=100


In [8]:
doc_table = soup.find_all(class_ = "blueRow")
# print(doc_table)

In [9]:
data = soup.find_all(class_='blueRow')

list_10k = []

for i, row in enumerate(data): 
    for a in data[i].find_all('a', href=True):
        url = a['href']
        if (url.startswith('/Archives/edgar/')):
            list_10k.append("https://www.sec.gov"+ url)
            print("https://www.sec.gov"+ url)

https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/0001564590-21-004599-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459020004475/0001564590-20-004475-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/0001564590-18-002956-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/0001564590-16-013195-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/0001193125-14-069681-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/0001193125-12-137560-index.htm
https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847-index.htm


In [10]:
print(list_10k)

['https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/0001564590-21-004599-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459020004475/0001564590-20-004475-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459018002956/0001564590-18-002956-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000156459016013195/0001564590-16-013195-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312514069681/0001193125-14-069681-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312512137560/0001193125-12-137560-index.htm', 'https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847-index.htm']


In [11]:
response0 = requests.get(url = list_10k[0], headers=header)

soup10k = BeautifulSoup(response0.content, 'html.parser')
print(response0)
print(response0.url)

<Response [200]>
https://www.sec.gov/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796-index.htm


In [15]:
report10k = []

for a in soup10k.find_all('a', href=True):
    url = a['href']
    print(url)
    if (url.find('10k') != -1):
        report10k.append("https://www.sec.gov"+ url)
        print("https://www.sec.gov"+ url)

/index.htm
/cgi-bin/browse-edgar?action=getcurrent
javascript:history.back()
/index.htm
/edgar/searchedgar/webusers.htm
/index.htm
/edgar/searchedgar/webusers.htm
/edgar/searchedgar/companysearch.html
/cgi-bin/viewer?action=view&cik=1318605&accession_number=0000950170-22-000796&xbrl_type=v
/ix?doc=/Archives/edgar/data/1318605/000095017022000796/tsla-20211231.htm
/Archives/edgar/data/1318605/000095017022000796/tsla-ex21_1.htm
/Archives/edgar/data/1318605/000095017022000796/tsla-ex23_1.htm
/Archives/edgar/data/1318605/000095017022000796/tsla-ex31_1.htm
/Archives/edgar/data/1318605/000095017022000796/tsla-ex31_2.htm
/Archives/edgar/data/1318605/000095017022000796/tsla-ex32_1.htm
/Archives/edgar/data/1318605/000095017022000796/img96779317_0.jpg
/Archives/edgar/data/1318605/000095017022000796/0000950170-22-000796.txt
/Archives/edgar/data/1318605/000095017022000796/tsla-20211231_def.xml
/Archives/edgar/data/1318605/000095017022000796/tsla-20211231_pre.xml
/Archives/edgar/data/1318605/0000950

In [14]:
print(report10k)

[]


In [13]:
# Find all of the text between paragraph tags and strip out the html
response1 = requests.get(url = report10k[0], headers=header)

soupText = BeautifulSoup(response1.content, 'html.parser')
print(response1)
print(response1.url)


IndexError: list index out of range

In [ ]:
page = soupText.find_all('p').getText()

In [ ]:
text = ""
for data in soupText.find_all("p"): 
    text += data.get_text()
    # print(data.get_text()) 

In [ ]:
# print(text)

In [ ]:
import unicodedata

text = unicodedata.normalize('NFKD', text.replace("\'", "'").replace("\ in\ form", " inform").replace("\n", " ").lower().strip())

In [ ]:
# print(text)
dev_docs = [doc for doc in text.split('\n')] 

In [ ]:
from   sklearn.feature_extraction.text import TfidfVectorizer
from   nltk.corpus import stopwords
import string

# Making stopwords list
stoplist = stopwords.words('english')
for el in [i for i in string.punctuation]:
    stoplist.append(el)

# Set up vectorizer
vectorizer = TfidfVectorizer(
    encoding='utf-8',
    min_df=1, # include words that occur in as few as a single document, 1 = a single document
    max_df=1.0, # include words that occur in as many as all documents, 1.0 = 100% = all documents
     binary=False,
    norm='l2',
    stop_words = stoplist,
    use_idf=True
)

X = vectorizer.fit_transform(dev_docs)

In [ ]:
print(X.shape)

In [ ]:
print(X)